In [10]:
import pandas as pd
from sklearn.utils.validation import column_or_1d

measurements = pd.read_csv('/data/public/MLA/VCHAMPS/measurements_train.csv').iloc[:,1:]

In [7]:
readmission = pd.read_csv('/home/daisy/FDA_Dataset/inpatient_all_final_1.csv').iloc[:,1:]

In [8]:
inpatient = pd.read_csv('/home/hassan/lily/MLA/FDA/inpatient_full_simple.csv').iloc[:,1:]
inpatient.head()

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,Died,AO,CVD,unique_admitting_specialty,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,1,3,14.16,2,0,0,0,0,0,3,...,0.6,3,0,0,0,0,0,2352.34,2016-07-22 21:36:24.0,0
1,2,21,71.17,5,2,10,0,0,1,9,...,1.5,10,1,0,9,1,1,0.00,2023-09-12 02:28:10.0,0
2,3,1,1.83,1,0,1,0,0,1,1,...,1.0,0,0,0,1,0,0,885.57,2002-03-30 18:12:14.0,0
3,4,1,7.15,1,0,1,0,0,1,1,...,1.0,0,0,0,1,0,0,0.00,2015-02-23 09:46:15.0,0
4,5,1,1.04,1,0,1,0,0,1,1,...,1.0,1,0,0,0,0,0,26.75,2004-10-02 06:54:56.0,1


In [9]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date

,Internalpatientid,Discharge date
0,1,2016-07-22 21:36:24.0
1,2,2023-09-12 02:28:10.0
2,3,2002-03-30 18:12:14.0
3,4,2015-02-23 09:46:15.0
4,5,2004-10-02 06:54:56.0
...,...,...
84531,169055,2002-04-12 16:51:53.0
84532,169057,2025-02-12 17:23:04.0
84533,169060,2017-02-08 09:55:03.0
84534,169062,2005-06-04 03:34:07.0


In [11]:
# Filter procedure date that's before discharge date
measurements_merged = inpatient_date.merge(measurements, how = 'left', on = 'Internalpatientid')
measurements_merged = measurements_merged.loc[measurements_merged['Measurement date'] <= measurements_merged['Discharge date']]
measurements_merged

,Internalpatientid,Discharge date,Age at measurement,Measurement date,Measurement,Result numeric,Result textual,State
0,1,2016-07-22 21:36:24.0,59.124538,2003-05-21 00:27:01.0,Temperature,95.804066,NaN,Indiana
1,1,2016-07-22 21:36:24.0,59.124538,2003-05-21 00:27:01.0,Pain,3.000000,NaN,Indiana
2,1,2016-07-22 21:36:24.0,59.124538,2003-05-21 00:27:01.0,Pulse,74.000000,NaN,Indiana
3,1,2016-07-22 21:36:24.0,59.124538,2003-05-21 00:27:01.0,Respiratory rate,16.000000,NaN,Indiana
4,1,2016-07-22 21:36:24.0,59.124538,2003-05-21 00:27:01.0,Blood pressure,NaN,140/96,Indiana
...,...,...,...,...,...,...,...,...
116415449,169064,2008-08-26 07:34:10.0,81.679966,2008-08-22 00:39:57.0,Temperature,98.787744,NaN,Wyoming
116415450,169064,2008-08-26 07:34:10.0,81.679966,2008-08-22 00:39:57.0,Pain,0.000000,NaN,Wyoming
116415451,169064,2008-08-26 07:34:10.0,81.679966,2008-08-22 00:39:57.0,Respiratory rate,18.000000,NaN,Wyoming
116415452,169064,2008-08-26 07:34:10.0,81.679966,2008-08-22 00:39:57.0,Blood pressure,NaN,103/56,Wyoming


: 